In [ ]:
import sys
import os
import re
LWNN=os.path.abspath('..')
if(not os.path.isfile('%s/Console.bat'%(LWNN))):
    LWNN=os.path.abspath('.')
p=os.path.abspath('%s/tools'%(LWNN))
sys.path.append(p)
import numpy as np
import glob
from keras.models import Sequential, Model, load_model
from keras.layers import *
from keras2lwnn import *
from verifyoutput import *
%matplotlib inline

For the gtest Concat, the case concat_1 is the first one, so can run below command to generate the output raw of each layer of its LWNN model.
```sh
rm tmp -fr
lwnn_gtest.exe --gtest_filter=*CPU.ConcatQ8* -m 0 -d
```

In [ ]:
CASE='concat_1'

In [ ]:
MODEL = '%s/gtest/models/%s/%s.h5'%(LWNN,CASE,CASE)
INPUT = '%s/gtest/models/%s/golden/input.raw'%(LWNN,CASE)
TMP= '%s/tmp'%(LWNN)

In [ ]:
model = load_model(MODEL)

In [ ]:
shape=[-1]+[eval(str(d)) for d in model.input.shape[1:]]

In [ ]:
data = np.fromfile(INPUT, dtype=np.float32).reshape(shape)
if(data.shape[0] > 1):
    data = data[0].reshape(shape)

In [ ]:
reLoadName = re.compile(r'([^\s]+)_(\d+)_(\d+)')
for layer in model.layers[1:]:
    layer_model = Model(inputs=model.input, outputs=layer.output)
    outR = layer_model.predict(data)
    name=layer.name
    if('batch_normalization' in name):
        name = layer.input.name
        name = name.split('/')[0]
        if(reLoadName.search(name)):
            grp = reLoadName.search(name).groups()
            name = '_'.join(grp[:-1])
    rawL = glob.glob('%s/%s*%s*.raw'%(TMP,CASE,name))
    if(len(rawL) > 0):
        raw = rawL[0]
        if('q8' in raw):
            name += '-q8'
            out = np.fromfile(raw, dtype=np.int8)
        elif('q16' in raw):
            name += '-q16'
            out = np.fromfile(raw, dtype=np.int16)
        else:
            out = np.fromfile(raw, dtype=np.float32)
        if('-Q' in raw):
            Q = eval(raw[:-4].split('-Q')[-1])
            out = out/(2**Q)
            out = out.astype(np.float32)
            name += '-Q%s'%(Q)
        compare(outR, out, '%s-%s'%(layer.name,name))
    else:
        print('skip', layer.name)